# Data Exploration and Preprocessing

In this notebook, we will load the dataset, explore it, and perform necessary preprocessing tasks.

In [48]:
# Import necessary libraries
import pandas as pd
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit learn libraries
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

## Load the Data

We will load the data from postgres database into a pandas DataFrame.

In [18]:
# Define a dictionary with the connection details
db_config = {
    'username': 'postgres',
    'password': 'pass123',
    'host': 'localhost',
    'port': '5432',
    'database': 'telecom'
}

# Create a connection to the PostgreSQL database
engine = create_engine(f"postgresql://{db_config['username']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}")

# Execute a SQL query and load the result into a DataFrame
query = text("SELECT * FROM xdr_data")
with engine.connect() as conn: 
    df = pd.read_sql_query(query, conn)

## Data Overview

In [34]:
# Print the first few rows of the DataFrame
df.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,7.349883e+18,4/23/2019 13:56,642.0,4/25/2019 2:06,53.0,130220.0,2.082003e+14,3.368112e+10,3.572961e+13,T10006B,...,21500768.0,12101768.0,12226975.0,16511157.0,340225685.0,6951608.0,483022848.0,13524725.0,50864502.0,388912579.0
1,1.311448e+19,4/23/2019 14:00,501.0,4/24/2019 23:17,453.0,119804.0,2.082014e+14,3.369966e+10,3.561431e+13,T92938B,...,7277265.0,14808877.0,5404356.0,17848389.0,281827903.0,3255023.0,746150700.0,1475289.0,38549811.0,305633791.0
2,1.311448e+19,4/23/2019 14:00,507.0,4/24/2019 23:17,453.0,119781.0,2.082014e+14,3.369966e+10,3.561431e+13,T92938B,...,10232699.0,8551438.0,8528398.0,19632728.0,73444975.0,8167950.0,787793727.0,10773216.0,49161715.0,105009018.0
3,1.311448e+19,4/23/2019 14:00,744.0,4/25/2019 0:24,330.0,123787.0,2.082017e+14,3.362404e+10,8.629340e+13,D21059A,...,15007252.0,16673992.0,14089357.0,6924105.0,174894914.0,41507.0,345556220.0,837674.0,28422142.0,209975300.0
4,1.311448e+19,4/23/2019 14:01,369.0,4/25/2019 4:13,905.0,137523.0,2.082017e+14,3.366612e+10,3.538111e+13,D42243B,...,9633967.0,17836471.0,22432445.0,11400.0,623985913.0,10539036.0,706817241.0,11317378.0,43951635.0,661274286.0


In [35]:
# Print the shape of the DataFrame
df.shape

(150001, 55)

In [36]:
# Check the DataFrame's information
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

In [37]:
# Describe the DataFrame
df.describe()

,Bearer Id,Start ms,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
count,1.490100e+05,150000.000000,150000.000000,1.500000e+05,1.494310e+05,1.489350e+05,1.494290e+05,122172.000000,122189.000000,150000.000000,...,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500000e+05,1.500000e+05
mean,1.013887e+19,499.188200,498.800880,1.046086e+05,2.082016e+14,4.188282e+10,4.847455e+13,109.795706,17.662883,13300.045927,...,1.163407e+07,1.100941e+07,1.162685e+07,1.100175e+07,4.220447e+08,8.288398e+06,4.211005e+08,8.264799e+06,4.112121e+07,4.546434e+08
std,2.893173e+18,288.611834,288.097653,8.103762e+04,2.148809e+10,2.447443e+12,2.241637e+13,619.782739,84.793524,23971.878541,...,6.710569e+06,6.345423e+06,6.725218e+06,6.359490e+06,2.439675e+08,4.782700e+06,2.432050e+08,4.769004e+06,1.127639e+07,2.441429e+08
min,6.917538e+18,0.000000,0.000000,7.142000e+03,2.040471e+14,3.360100e+10,4.400152e+11,0.000000,0.000000,0.000000,...,5.300000e+01,1.050000e+02,4.200000e+01,3.500000e+01,2.516000e+03,5.900000e+01,3.290000e+03,1.480000e+02,2.866892e+06,7.114041e+06
25%,7.349883e+18,250.000000,251.000000,5.744050e+04,2.082014e+14,3.365130e+10,3.546071e+13,32.000000,2.000000,43.000000,...,5.833501e+06,5.517965e+06,5.777156e+06,5.475981e+06,2.104733e+08,4.128476e+06,2.101869e+08,4.145943e+06,3.322201e+07,2.431068e+08
50%,7.349883e+18,499.000000,500.000000,8.639900e+04,2.082015e+14,3.366371e+10,3.572201e+13,45.000000,5.000000,63.000000,...,1.161602e+07,1.101345e+07,1.164222e+07,1.099638e+07,4.234081e+08,8.291208e+06,4.218030e+08,8.267071e+06,4.114331e+07,4.558411e+08
75%,1.304243e+19,749.000000,750.000000,1.324302e+05,2.082018e+14,3.368349e+10,8.611970e+13,70.000000,15.000000,19710.750000,...,1.744852e+07,1.651556e+07,1.747048e+07,1.650727e+07,6.331742e+08,1.243162e+07,6.316918e+08,1.238415e+07,4.903424e+07,6.657055e+08
max,1.318654e+19,999.000000,999.000000,1.859336e+06,2.140743e+14,8.823971e+14,9.900120e+13,96923.000000,7120.000000,378160.000000,...,2.325910e+07,2.201196e+07,2.325919e+07,2.201196e+07,8.434419e+08,1.655879e+07,8.434425e+08,1.655882e+07,7.833131e+07,9.029696e+08


In [38]:
# Check for missing values
df.isnull().sum()

Bearer Id                                      991
Start                                            1
Start ms                                         1
End                                              1
End ms                                           1
Dur. (ms)                                        1
IMSI                                           570
MSISDN/Number                                 1066
IMEI                                           572
Last Location Name                            1153
Avg RTT DL (ms)                              27829
Avg RTT UL (ms)                              27812
Avg Bearer TP DL (kbps)                          1
Avg Bearer TP UL (kbps)                          1
TCP DL Retrans. Vol (Bytes)                  88146
TCP UL Retrans. Vol (Bytes)                  96649
DL TP < 50 Kbps (%)                            754
50 Kbps < DL TP < 250 Kbps (%)                 754
250 Kbps < DL TP < 1 Mbps (%)                  754
DL TP > 1 Mbps (%)             

In [39]:
# Check for duplicates
df.duplicated().sum()

0

## User Overview analysis

An overview of the user's behavior on the applications


### Top 10 handsets used by the customers

In [40]:
# Identify the top 10 handsets used by the customers
top_10_handsets = df['Handset Type'].value_counts().head(10)
print(top_10_handsets)

Huawei B528S-23A                19752
Apple iPhone 6S (A1688)          9419
Apple iPhone 6 (A1586)           9023
undefined                        8987
Apple iPhone 7 (A1778)           6326
Apple iPhone Se (A1723)          5187
Apple iPhone 8 (A1905)           4993
Apple iPhone Xr (A2105)          4568
Samsung Galaxy S8 (Sm-G950F)     4520
Apple iPhone X (A1901)           3813
Name: Handset Type, dtype: int64


### Top 3 handset manufacturers

In [42]:
# Identify the top 3 handset manufacturers
top_3_manufacturers = df['Handset Manufacturer'].value_counts().head(3)
print(top_3_manufacturers)

Apple      59565
Samsung    40839
Huawei     34423
Name: Handset Manufacturer, dtype: int64


In [43]:
# Identify the top 5 handsets per top 3 handset manufacturer
top_5_handsets_per_manufacturer = df[df['Handset Manufacturer'].isin(top_3_manufacturers.index)].groupby('Handset Manufacturer')['Handset Type'].value_counts().groupby(level=0).head(5)
print(top_5_handsets_per_manufacturer)

Handset Manufacturer  Handset Type                  
Apple                 Apple iPhone 6S (A1688)            9419
                      Apple iPhone 6 (A1586)             9023
                      Apple iPhone 7 (A1778)             6326
                      Apple iPhone Se (A1723)            5187
                      Apple iPhone 8 (A1905)             4993
Huawei                Huawei B528S-23A                  19752
                      Huawei E5180                       2079
                      Huawei P20 Lite Huawei Nova 3E     2021
                      Huawei P20                         1480
                      Huawei Y6 2018                      997
Samsung               Samsung Galaxy S8 (Sm-G950F)       4520
                      Samsung Galaxy A5 Sm-A520F         3724
                      Samsung Galaxy J5 (Sm-J530)        3696
                      Samsung Galaxy J3 (Sm-J330)        3484
                      Samsung Galaxy S7 (Sm-G930X)       3199
Name: Handset Typ

In [44]:
def preprocess_data(df, numeric_features, categorical_features):

    # Define preprocessing for numeric columns (replace missing values -> scale values)
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())])

    # Define preprocessing for categorical columns (replace missing values -> one hot encoding)
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    # Combine preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])

    # Create preprocessing and training pipeline
    pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

    # Now you can fit the pipeline to your data and use it to transform your data
    pipeline.fit(df)
    X = pipeline.transform(df)
    
    return X

In [45]:
numeric_features = ['Dur. (ms).1', 'Total DL (Bytes)', 'Total UL (Bytes)']
categorical_features = ['Handset Manufacturer', 'Handset Type']
X = preprocess_data(df, numeric_features, categorical_features)

In [47]:
# Convert the transformed data back to a DataFrame
X_df = pd.DataFrame(X)

# Display the first few rows of the DataFrame
print(X_df.head())

                                                   0
0    (0, 0)\t0.3160443119960812\n  (0, 1)\t-0.269...
1    (0, 0)\t0.18751722150112954\n  (0, 1)\t-0.61...
2    (0, 0)\t0.18723332674008955\n  (0, 1)\t-1.43...
3    (0, 0)\t0.2366630498824108\n  (0, 1)\t-1.002...
4    (0, 0)\t0.40616510338616235\n  (0, 1)\t0.846...
